# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#Abrir o navegador
navegador = webdriver.Chrome()

# pesquisar cotação dolar
navegador.get("https://www.google.com/")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# pesquisar cotação euro
navegador.get("https://www.google.com/")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# pesquisar cotação ouro
navegador.get("https://www.melhorcambio.com/")
aba_original = navegador.window_handles[0]
navegador.find_element_by_xpath('//*[@id="commodity-hoje"]/tbody/tr[2]/td[2]/a/img').click()
aba_nova = navegador.window_handles[1]
navegador.switch_to.window(aba_nova)
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')
print(cotacao_ouro)
navegador.quit()

5.3382
6.465227475000001
304.29


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [34]:
import pandas as pd

Tabela_Produtos = pd.read_excel('Produtos.xlsx')
display(Tabela_Produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [35]:
Tabela_Produtos.loc[Tabela_Produtos["Moeda"]=="Dólar","Cotação"] = float(cotacao_dolar)
Tabela_Produtos.loc[Tabela_Produtos["Moeda"]=="Euro","Cotação"] = float(cotacao_euro)
Tabela_Produtos.loc[Tabela_Produtos["Moeda"]=="Ouro","Cotação"] = float(cotacao_ouro)

Tabela_Produtos['Preço Base Reais'] = Tabela_Produtos['Preço Base Original'] * Tabela_Produtos['Cotação']
Tabela_Produtos['Preço Final'] = Tabela_Produtos['Ajuste'] * Tabela_Produtos['Preço Base Reais']
Tabela_Produtos['Preço Final'] = Tabela_Produtos['Preço Final'].map('{:.2f}'.format)
display(Tabela_Produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.338200,5338.146618,1.40,7473.41
1,Carro Renault,4500.00,Euro,6.465227,29093.523638,2.00,58187.05
2,Notebook Dell,899.99,Dólar,5.338200,4804.326618,1.70,8167.36
3,IPhone,799.00,Dólar,5.338200,4265.221800,1.70,7250.88
4,Carro Fiat,3000.00,Euro,6.465227,19395.682425,1.90,36851.80
5,Celular Xiaomi,480.48,Dólar,5.338200,2564.898336,2.00,5129.80
6,Joia 20g,20.00,Ouro,304.290000,6085.800000,1.15,6998.67


### Agora vamos exportar a nova base de preços atualizada

In [36]:
Tabela_Produtos.to_excel('Produtos Atualizados.xlsx')